In [5]:
#====================================================================================
#Retrieve and save the latest set of extraction files for a named TRTH schedule.
#Display contents of the notes files.

#Pre-requisite: a pre-existing schedule (it can be created manually, or using the API).

#Typical use case: regular retrieval of extraction files generated by a recurring schedule.
#Example:
# - A manually created daily schedule is set to run every day at 1 am.
# - This sample runs every day at 3 am to retrieve the generated files.

#The extraction file set typically contains:
# - The "Full" file, containing the requested data
# - The "Note" file, containing extraction notes, warnings and errors
# - The "RicMaintenanceNote" file, containing information on RIC name changes;
#   it is generated (or not) based on a setting in the user preferences, in the GUI.

#The extraction file compression is defined in the report template.
#It can be:
# - None: delivers a text file with extension .csv
# - ZIP:  delivers a zipped text file with extension .zip
# - GZIP: delivers a gzipped text file with extension .gz
#The selected compression is applied to the "Full" and "RicMaintenanceNote" files.
#The "Note" file is always a text file with extension .txt
#When creating a report template with the:
# - API: default compression settings are applied, based on the template type.
# - GUI: settings are done manually, with restictions based on the template type.
#This demo code handles all these compression settings.
#====================================================================================

#Set these parameters before running the code:
filePath = "/Users/u8007966/Downloads/"  #Location to save downloaded files
myScheduleName = "myScheduleName"  #Name of an existing schedule, for which we want to retrieve data
myUsername = "myUsername"
myPassword = "myPassword"
useAws = True
#Set the last parameter above to:
# - False to download from TRTH servers
# - True to download from Amazon Web Services cloud (recommended, it is faster)

#Constants:
baseUrl = "https://hosted.datascopeapi.reuters.com/RestApi/v1/"
requestHeaders = { "Prefer":"respond-async", "Content-Type":"application/json" }

#Imports:
import sys
import requests
import json
import shutil
import gzip
import zipfile

#====================================================================================
#Functions list:
# - Get authentication token
# - Get schedule ID (from name)
# - Get report extraction ID for last extraction (from schedule ID)
# - Get report file list, with for each file its ID, type, name and size (from report extraction ID)
# - Download and save a file to disk (using the file ID)
# - Download and display the extraction notes file (using the file ID), no save to disk
# - Read and display a text file
# - Read, decompress and display a zipped text file
# - Read, decompress and display a gzipped text file
#====================================================================================

#Get authentication token
def getAuthenticationToken( userName, password ):
    if not (userName and password):
        print ('ERROR: missing username or password')
        sys.exit()
    authReqUrl = baseUrl + "Authentication/RequestToken"
    credentialsData = { 'Credentials' : { 'Username' : userName, 'Password' : password} }
    r = requests.post( url = authReqUrl, json = credentialsData, headers = requestHeaders )
    if (r.status_code == 200):
        jsonResponse = json.loads(r.text.encode('ascii', 'ignore'))
        token = jsonResponse["value"]
        return token
    else:
        print ("ERROR: could not retrieve authentication token, check username and password:")
        print ("Status code: ", r.status_code, "\nResponse:\n", r.text)
        sys.exit()

#Get schedule ID (from name)
def getScheduleId( token, scheduleName ):
    getScheduleIdUrl = baseUrl + "Extractions/ScheduleGetByName(ScheduleName="
    getScheduleIdUrl = getScheduleIdUrl + "'" + scheduleName + "')"
    requestHeaders["Authorization"] = "token " + token
    r = requests.get( getScheduleIdUrl, headers=requestHeaders )
    if (r.status_code==200):
        jsonResponse = json.loads(r.text.encode('ascii', 'ignore'))
        return jsonResponse["ScheduleId"]
    else:
        print ("ERROR: could not retrieve schedule id for schedule name: ", scheduleName)
        print ("Status code: ", r.status_code, "\nResponse:\n", r.text)
        sys.exit()

#Get report extraction ID for last extraction (from schedule ID)
def getReportExtractionId( token, scheduleId ):
    getScheduleUrl = baseUrl + "Extractions/Schedules"
    getScheduleUrl = getScheduleUrl + "('" + scheduleId + "')/LastExtraction"
    requestHeaders["Authorization"] = "token " + token
    r = requests.get( getScheduleUrl, headers=requestHeaders )
    if (r.status_code==200):
        jsonResponse = json.loads(r.text.encode('ascii', 'ignore'))
        return jsonResponse["ReportExtractionId"]
    else:
        print ("ERROR: could not retrieve report id:")
        print ("Status code: ", r.status_code, "\nResponse:\n", r.text)
        if (r.status_code==204):
            print ("The schedule has never run")
        sys.exit()

#Get report file list, with for each file its ID, type, name and size (from report extraction ID)
def getReportFiles( token, extractionId ):
    getReportFilesUrl = baseUrl + "Extractions/ReportExtractions"
    getReportFilesUrl = getReportFilesUrl + "('" + extractionId + "')/Files"
    requestHeaders["Authorization"] = "token " + token
    r = requests.get( getReportFilesUrl, headers=requestHeaders )
    if (r.status_code == 200):
        value = json.loads(r.text.encode('ascii', 'ignore'))["value"]
        i = 0
        allFilesData = []
        for fileObject in value:
            oneFileData=[value[i]["ExtractedFileId"],value[i]["FileType"],value[i]["ExtractedFileName"],value[i]["Size"]]
            allFilesData.append(oneFileData)
            i += 1
        return allFilesData
    else:
        print ("ERROR: could not retrieve the file list:")
        print ("Status code: ", r.status_code, "\nResponse:\n", r.text)
        sys.exit()

#Download and save a file to disk (using the file ID)
def downloadAndSaveFile( token, fileId, fullFileName, decode, useAws ):
    fileDownloadUrl = baseUrl + "Extractions/ExtractedFiles"
    fileDownloadUrl = fileDownloadUrl + "('" + fileId + "')/$value"
    requestHeaders["Authorization"] = "token " + token
    if (useAws):
        requestHeaders["X-Direct-Download"] = "true"
    r = requests.get( fileDownloadUrl, headers=requestHeaders, stream=True )
    #For debugging:
    if useAws:  #AWS does not set header Content-Encoding="gzip".
        print ('Content response headers (AWS server): type: ' + r.headers["Content-Type"])
    else:
        print ('Content response headers (TRTH server): type: ' + r.headers["Content-Type"] + ' - encoding: ' + r.headers["Content-Encoding"])
    r.raw.decode_content = decode
    rr = r.raw
    chunk_size = 1024
    try:
        with open(fullFileName, 'wb') as fd:
            shutil.copyfileobj(rr, fd, chunk_size)    
        fd.close
    except:
        print ("ERROR: could not save the file, does the directory exist ?")
        sys.exit()

#Download and display the extraction notes file (using the file ID), no save to disk
def displayExtractionNotesFile( token, fileId ):
    fileDownloadUrl = baseUrl + "Extractions/ExtractedFiles"
    fileDownloadUrl = fileDownloadUrl + "('" + fileId + "')/$value"
    requestHeaders["Authorization"] = "token " + token
    r = requests.get( fileDownloadUrl, headers=requestHeaders, stream=True )
    #For debugging:
    #print ('Content response headers (TRTH server): type: ' + r.headers["Content-Type"] + ' - encoding: ' + r.headers["Content-Encoding"])
    print (r.text)

#Read and display a text file
def readAndDisplayTextFile( fullTextFileName ):
    text = ""
    try:
        with open(fullTextFileName, 'r') as fd:
            for line in fd:
                text = text + line;
        print(text)
        fd.close
    except:
        print ("ERROR: could not read the file, does it exist ?")
        sys.exit()

#Read, decompress and display a zipped text file
def readDecompressAndDisplayZippedTextFile( fullZipFileName, contentFileName ):
    try:
        with zipfile.ZipFile(fullZipFileName) as zipedTextFile:
            with zipedTextFile.open(contentFileName) as textFile:
                print(textFile.read().decode("utf-8"))
    except:
        print ("ERROR: could not read the file, does it exist ?")
        sys.exit()

#Read, decompress and display a gzipped text file
def readDecompressAndDisplayGzippedTextFile( fullGzipFileName, contentFileName ):    
    try:
        with gzip.open(fullGzipFileName, 'rb') as gzipedTextFile:
            print(gzipedTextFile.read().decode("utf-8"))
    except:
        print ("ERROR: could not read the file, does it exist ?")
        sys.exit()

#====================================================================================
# Main
#====================================================================================
if __name__ == "__main__":
    try:
        token = getAuthenticationToken( myUsername, myPassword )
        print ("Token:\n", token, "\n")

        scheduleId = getScheduleId( token, myScheduleName )
        print ("Retrieved schedule ID: ", scheduleId, " from schedule name: ", myScheduleName,"\n")

        reportExtractionId = getReportExtractionId( token, scheduleId )
        print ("Retrieved report extraction ID: ", reportExtractionId,"\n")

        extractedFilesList = getReportFiles( token, reportExtractionId )
        print ("Retrieved file list. There are ", str(len(extractedFilesList)), " files:")

        for fileItem in extractedFilesList:
            fileId = fileItem[0]
            fileType = fileItem[1]
            fileName = fileItem[2]
            fileSize = fileItem[3]
            fullFileName = filePath + fileName
            print ("\n=============================================================================")
            print("Name: ", fileName)
            print("Type: ", fileType, " - Size: ", str(fileSize), " - ID: " + fileId)

            if (fileSize==0):
                print("INFO: File is empty, ignore it.")
            else:
                if (fileType=="Full"):
                    print ('Saving data file, in directory: ' + filePath + '\n... please be patient')
                    downloadAndSaveFile( token, fileId, fullFileName, True, useAws )
                    #Alternative to override the template setting, and gzip a CSV delivered file:                
                    #if (fileName[-3:]=="csv"):    #file name ends in .csv
                    #    fullFileName = fullFileName + ".gz"
                    #    downloadAndSaveFile( token, fileId, fullFileName, False, useAws )
                    #else:
                    #    downloadAndSaveFile( token, fileId, fullFileName, True, useAws )
                    print ("Data saved to file:\n", fullFileName)

                elif (fileType=="Note"):
                    print ('Saving extraction notes file, in directory: ' + filePath + '\n... this should be quick')
                    downloadAndSaveFile( token, fileId, fullFileName, True, False )
                    print ("Extraction notes saved to text file.\n")
                    #Read extraction notes from saved file (it is always a text file):
                    print ("Extraction notes read from text file:")
                    print ("-------------------------------------")
                    readAndDisplayTextFile(fullFileName)
                    #Alternative: display extraction notes directly:
                    print ("Extraction notes retrieved directly from the server:")
                    print ("----------------------------------------------------")
                    displayExtractionNotesFile( token, fileId )

                elif (fileType=="RicMaintenanceNote"):
                    print ('Saving Ric maintenance notes to file, in directory: ' + filePath + '\n... this should be quick')
                    downloadAndSaveFile( token, fileId, fullFileName, True, False )
                    print ("Ric maintenance notes saved to file.\n")
                    #Read RIC maintenance notes from saved file (it can be text or compressed):
                    print ("Ric maintenance notes read from file:")
                    print ("-----------------------------------------")
                    if (fileName[-3:]=="csv"):    #file name ends in .csv
                        readAndDisplayTextFile(fullFileName)
                    elif (fileName[-3:]=="zip"):
                        contentFileName = fileName.strip(".zip")
                        readDecompressAndDisplayZippedTextFile( fullFileName, contentFileName )
                    elif (fileName[-3:]==".gz"):
                        readDecompressAndDisplayGzippedTextFile( fullFileName, contentFileName )
                    else:
                        print ("Unsupported file extension: ", fileName[-3:])
                else:
                    print ("Unknown file type: ", fileType)
    except:
        print ("\nERROR: Program terminated abnormally.")

Token:
 _Ss7_ZS0hst7biHu1wXWBbWCeBi4SrrmmWQjxZd3P4VVP3s-H59ImJglDwodPUaCs8PNOQo_QQ9F1eqwaAIYhnsm26pMHmlwiZLIORSity-k18B8QkGa_UieGKmC7K5XT7ddZjZVtyFtUjk89BkbbtVJCPgJCGPSk094Foy2dvpSBGYuEtxbA0GETYESGeIzd3ffQ7iKki7-PBHDtE9GsF3roUIryfhO1XdPAWN_1XiCdiuAqN5RzYdBeGvw8Zjh-YDncWsvF1LGBviyE7w2fBRZpX0Eb3P8XPEsZCYeL3yk 

Retrieved schedule ID:  0x05eccbe7487b3016  from schedule name:  Demo_TicksTrades_Schedule 

Retrieved report extraction ID:  2000000004063864 

Retrieved file list. There are  2  files:

Name:  33314.Demo_TicksTrades_Schedule.20171026.083240.2000000004063864.tm04n01.csv.gz.notes.txt
Type:  Note  - Size:  2651  - ID: VjF8MHgwNWVjY2MwMGViZWIyZjc2fA
Saving extraction notes file, in directory: /Users/u8007966/Downloads/
... this should be quick
Content response headers (TRTH server): type: text/plain - encoding: gzip
Extraction notes saved to text file.

Extraction notes read from text file:
-------------------------------------
Extraction Services Version 11.2.37366 (410ebb92dbb3), 